### Count the number of paths in a matrix with a given cost to reach the destination cell

Given an M x N matrix where each cell has a non-negative cost associated with it, count the number of paths to reach the lst cell (M-1, N-1) of the matrix from its first cell (0, 0) such that the path has a given cost. We can only move one unit right or one unit down from any cell, ie from cell (i, j) we can move to (i, j+1) or (i+1, j).

Example:

```
Consider the following 4x4 matrix where cost is 25. There are two paths with a cost of 25.

4     7     1     6

5     7     3     9

3     2     1     2

7     1     6     3

Either 4, 7, 1, 3, 1, 6, 3
or 4, 5, 7, 3, 1, 2, 3
```

In [1]:
def pathCost(matrix, cost, row=0, col=0, currCost=0):
    currCost += matrix[row][col]
    w = len(matrix[0])
    h = len(matrix)
    if currCost > cost:
        return 0
    if row == h-1 and col == w-1:
        return 1 if currCost == cost else 0
    right = 0
    down = 0
    if col < w-1:
        right = pathCost(matrix, cost, row, col+1, currCost)
    if row < h-1:
        down = pathCost(matrix, cost, row+1, col, currCost)
    return right + down

In [2]:
matrix = [
    [4,7,1,6],
    [5,7,3,9],
    [3,2,1,2],
    [7,1,6,3]
]

cost = 25

matrix2 = [
    [1,2,3,4,5,6,7],
    [1,2,3,4,5,6,7],
    [1,2,3,4,5,6,7],
    [1,2,3,4,5,6,7],
    [1,2,3,4,5,6,7],
    [1,2,3,4,5,6,7],
    [1,2,3,4,5,6,7]
]
cost2 = 55

In [3]:
%%time
pathCost(matrix, cost)

CPU times: user 33 µs, sys: 0 ns, total: 33 µs
Wall time: 36 µs


2

In [4]:
%%time
"""
Paths that add up to 29:
4,7,1,3,9,2,3
4,5,3,7,1,6,3
4,5,7,3,1,6,3
"""
pathCost(matrix, 29)


CPU times: user 101 µs, sys: 0 ns, total: 101 µs
Wall time: 106 µs


3

In [5]:
%%time
pathCost(matrix2, cost2)

CPU times: user 2.21 ms, sys: 27 µs, total: 2.23 ms
Wall time: 3.12 ms


51

### With a lookup table

With a lookup table, we can store the costs it takes to get to each cell.

Going straight across the top row and straight down the left column are easy to calculate.

From there we fill in the rest of the table - we can either get to the cell by coming from the top or from the left. Store all possibilities until you reach anything greater than the cost - at that point we can indicate it's not worth it so we don't waste our time any farther.

Add up how many paths we get of the desired cost in the final cell.

In [6]:
def pathCostLookup(matrix, cost):
    w = len(matrix[0])
    h = len(matrix)
    lookup = [[0] * w for _ in range(h)]
    lookup[0][0] = [matrix[0][0]]
    for i in range(1, w):
        lookup[0][i] = [lookup[0][i-1][0] + matrix[0][i]]
    for j in range(1, h):
        lookup[j][0] = [lookup[j-1][0][0] + matrix[j][0]]
    for row in range(1, h):
        for col in range(1, w):
            lookup[row][col] = []
            paths = lookup[row-1][col] + lookup[row][col-1]
            for path in paths:
                cell_cost = path + matrix[row][col]
                if (row < h-1 or col < w-1) and cell_cost < cost:
                    lookup[row][col].append(cell_cost)
                elif row == h-1 and col == w-1 and cell_cost == cost:
                    lookup[row][col].append(cell_cost)
    return len(lookup[-1][-1])

In [7]:
%%time
pathCostLookup(matrix, cost)

CPU times: user 53 µs, sys: 0 ns, total: 53 µs
Wall time: 57.2 µs


2

In [8]:
%%time
pathCostLookup(matrix2, cost2)

CPU times: user 1.85 ms, sys: 16 µs, total: 1.87 ms
Wall time: 1.88 ms


51

### Lookup dictionary, working backwards

We can also work backwards from the destination cell, subtracting the cell's cost from the given cost, and only keeping those paths that finish at 0 at the starting cell.

For each cell, keep the number of paths that can get us to that cell in the appropriate cost.

The key for the dictionary needs to track the position and the cost that we've counted down to at that point.

In [9]:
def pathCostDictionary(matrix, cost, row=None, col=None, lookup=None):
    if row == None:
        row = len(matrix) - 1
        col = len(matrix[0]) - 1
        lookup = {}
    if cost < 0:
        return 0
    if row == 0 and col == 0:
        return 1 if matrix[0][0] - cost == 0 else 0
    key = f'{row}-{col}-{cost}'
    if key not in lookup:
        if row == 0:
            lookup[key] = pathCostDictionary(matrix, cost-matrix[0][col], 0, col-1, lookup)
        elif col == 0:
            lookup[key] = pathCostDictionary(matrix, cost-matrix[row][0], row-1, 0, lookup)
        else:
            lookup[key] = pathCostDictionary(matrix, cost-matrix[row][col], row, col-1, lookup) + \
                           pathCostDictionary(matrix, cost-matrix[row][col], row-1, col, lookup)
    return lookup[key]
    

In [10]:
%%time
pathCostDictionary(matrix2, cost2)

CPU times: user 654 µs, sys: 20 µs, total: 674 µs
Wall time: 704 µs


51